# COMP90051 Project 2

In [1]:
# Do not edit. These are the only imports permitted.
%matplotlib inline
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

## 1. Implementing ε-Greedy and UCB

In [2]:
class MAB(ABC):
    """
    Abstract class that represents a multi-armed bandit (MAB)
    """
    
    @abstractmethod
    def play(self, tround, context):
        """
        Play a round
        
        Arguments
        =========
        tround : int
            positive integer identifying the round
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to the arms
        
        Returns
        =======
        arm : int
            the positive integer arm id for this round
        """
    
    @abstractmethod
    def update(self, arm, reward, context):
        """
        Updates the internal state of the MAB after a play
        
        Arguments
        =========
        arm : int
            a positive integer arm id in {1, ..., self.narms}
        
        reward : float
            reward received from arm
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to arms
        """

In [3]:
# Helper Functions

def select_with_uniformly_tie_breaking(arr):
    argmax_arr = np.argwhere(arr == np.amax(arr)).flatten()
    return argmax_arr[np.random.randint(argmax_arr.shape[0])]

In [4]:
class EpsGreedy(MAB):
    """
    Epsilon-Greedy multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    epsilon : float
        explore probability

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, epsilon, Q0=np.inf):
        self.narms = narms
        self.epsilon = epsilon
        self.Q0 = Q0
        self.times_pulled = np.zeros(narms)
        self.cumulative_rewards = np.zeros(narms)

        
    def play(self, tround, context=None):
        if np.random.binomial(1, self.epsilon):
            return np.random.randint(self.narms) + 1
        else:
            Q_table = np.zeros(self.narms)
            for i in range(self.narms):
                if self.times_pulled[i] == 0:
                    Q_table[i] = self.Q0
                else:
                    Q_table[i] = self.cumulative_rewards[i] / self.times_pulled[i]
            return select_with_uniformly_tie_breaking(Q_table) + 1

    def update(self, arm, reward, context=None):
        self.cumulative_rewards[arm - 1] += reward
        self.times_pulled[arm - 1] += 1
        

In [5]:
class UCB(MAB):
    """
    Upper Confidence Bound (UCB) multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    rho : float
        positive real explore-exploit parameter

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, rho, Q0=np.inf):
        self.narms = narms
        self.rho = rho
        self.Q0 = Q0
        self.times_pulled = np.zeros(narms)
        self.cumulative_rewards = np.zeros(narms)
    
    def play(self, tround, context=None):
        Q_table = np.zeros(self.narms)
        
        for i in range(self.narms):
            if self.times_pulled[i] == 0:
                Q_table[i] = self.Q0
            else:
                mu = self.cumulative_rewards[i] / self.times_pulled[i]
                ucb = np.sqrt(self.rho * np.log(tround) / self.times_pulled[i])
                Q_table[i] = mu + ucb

        return select_with_uniformly_tie_breaking(Q_table) + 1
        
    def update(self, arm, reward, context=None):
        self.cumulative_rewards[arm - 1] += reward
        self.times_pulled[arm - 1] += 1
        

## 2. The Basic Thompson Bandit

In [6]:
class BetaThompson(MAB):
    """
    Beta-Bernoulli Thompson sampling multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    alpha0 : float, optional
        positive real prior hyperparameter

    beta0 : float, optional
        positive real prior hyperparameter
    """
    
    def __init__(self, narms, alpha0=1.0, beta0=1.0):
        self.narms = narms
        self.alpha0 = alpha0
        self.beta0 = beta0
        self.success = np.zeros(narms)
        self.fail = np.zeros(narms)
        
    def play(self, tround, context=None):
        sampling_table = np.zeros(self.narms)
        for i in range(self.narms):
            sampling_table[i] = np.random.beta(self.alpha0+self.success[i], self.beta0+self.fail[i])

        return select_with_uniformly_tie_breaking(sampling_table) + 1
        
    def update(self, arm, reward, context=None):
        if reward:
            self.success[arm-1] += 1
        else:
            self.fail[arm-1] += 1

## 3. Off-Policy Evaluation

In [7]:
def offlineEvaluate(mab, arms, rewards, contexts, nrounds=None):
    """
    Offline evaluation of a multi-armed bandit
    
    Arguments
    =========
    mab : instance of MAB
    
    arms : 1D int array, shape (nevents,) 
        integer arm id for each event
    
    rewards : 1D float array, shape (nevents,)
        reward received for each event
    
    contexts : 2D float array, shape (nevents, mab.narms*nfeatures)
        contexts presented to the arms (stacked horizontally) 
        for each event.
        
    nrounds : int, optional
        number of matching events to evaluate `mab` on.
    
    Returns
    =======
    out : 1D float array
        rewards for the matching events
    """
    
    matching_rewards = []
    
    if nrounds != None:
        event_index = 0
        N_EVENTS = arms.shape[0]
        for t in range(1, nrounds+1):
            if (event_index >= N_EVENTS):
                break

            while True:
                mab_chosen_arm = mab.play(t, contexts[event_index])
                logging_arm = arms[event_index]
                if mab_chosen_arm == logging_arm:
                    reward = rewards[event_index]
                    matching_rewards.append(reward)
                    mab.update(mab_chosen_arm, reward, contexts[event_index])
                    event_index += 1
                    break
                    
                event_index += 1
                if event_index >= N_EVENTS:
                    break
                
    return np.array(matching_rewards)

In [8]:
with open(r"./dataset.txt", 'r') as fhand:
    events = np.array([line.strip().split(" ") for line in fhand]).astype("int32")
    arms, rewards, contexts = np.hsplit(events, [1,2])
    arms = arms.flatten()
    rewards = rewards.flatten()

In [9]:
mab = EpsGreedy(10, 0.05)
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 8000)
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

EpsGreedy average reward 0.13811007268951195


In [10]:
mab = UCB(10, 1.0)
results_UCB = offlineEvaluate(mab, arms, rewards, contexts, 8000)
print('UCB average reward', np.mean(results_UCB))

UCB average reward 0.16040609137055836


In [11]:
mab = BetaThompson(10, 1.0, 1.0)
results_BetaThompson = offlineEvaluate(mab, arms, rewards, contexts, 8000)
print('BetaThompson average reward', np.mean(results_BetaThompson))

BetaThompson average reward 0.19447287615148415


## 4. Contextual Bandits - LinUCB

In [12]:
class LinUCB(MAB):
    """
    Contextual multi-armed bandit (LinUCB)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    alpha : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, alpha):
        self.narms = narms
        self.ndims = ndims
        self.alpha = alpha
        self.A = np.stack([np.identity(ndims) for _ in range(narms)], axis=0)
        self.b = np.zeros((narms, ndims, 1))
        
    def play(self, tround, context):
        context = context.reshape((self.narms, self.ndims))
        
        p_arr = np.zeros(self.narms)
        for i in range (self.narms):
            feature = context[i].reshape(self.ndims, 1) # shape(ndims, 1)
            A_i_inv = np.linalg.inv(self.A[i])
            theta_i = A_i_inv @ self.b[i] # shape(ndims, 1)
            p_arr[i] = theta_i.T@feature + self.alpha*np.sqrt(feature.T@A_i_inv@feature)
            
        return select_with_uniformly_tie_breaking(p_arr) + 1
    
    def update(self, arm, reward, context):
        feature = context.reshape((self.narms, self.ndims))[arm-1].reshape((self.ndims,1))
        self.A[arm-1] += feature @ feature.T
        self.b[arm-1] += reward * feature
    

In [13]:
mab = LinUCB(10, 10, 1.0)
results_LinUCB = offlineEvaluate(mab, arms, rewards, contexts, 8000)
print('LinUCB average reward', np.mean(results_LinUCB))

LinUCB average reward 0.5632530120481928


## 5. Contextual Bandits - LinThompson

In [14]:
class LinThompson(MAB):
    """
    Contextual Thompson sampled multi-armed bandit (LinThompson)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    v : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, v):
        self.narms = narms
        self.ndims = ndims
        self.v = v
        self.mu_mean = np.zeros(ndims)
        self.f = np.zeros(ndims)
        self.B = np.identity(ndims)
        
        
    def play(self, tround, context):
        mu_sampled = np.random.multivariate_normal(self.mu_mean, self.v**2 * np.linalg.inv(self.B))
        context = context.reshape((self.narms, self.ndims))
        
        rewards_table = np.zeros(self.narms)
        for i in range(self.narms):
            feature = context[i].reshape((self.ndims,1))
            rewards_table[i] = feature.T @ mu_sampled
        
        return select_with_uniformly_tie_breaking(rewards_table) + 1
            
    
    def update(self, arm, reward, context):
        feature = context.reshape((self.narms, self.ndims))[arm-1].reshape((self.ndims, 1))
        self.B += feature @ feature.T
        self.f += reward * feature.ravel()
        self.mu_mean += np.linalg.inv(self.B) @ self.f
        

In [15]:
mab = LinThompson(10, 10, 1.0)
results_LinThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinThompson average reward', np.mean(results_LinThompson))

LinThompson average reward 0.44


## 6. Evaluation
### 6.A.

### 6.B.